In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
%matplotlib notebook

In [ ]:
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['axes.grid'] = True

In [ ]:
from ukr.un import UNHR

In [ ]:
unhr = UNHR()

In [ ]:
unhr.update(store=False)

In [ ]:
last_date, last_stat = unhr.last()
print(f'{last_date.date()}: total {last_stat["killed"].total:,} KILLED and {last_stat["injured"].total:,} INJURED')

In [ ]:
import seaborn as sns

In [ ]:
cum = unhr.data.dropna().stack().stack().reset_index()
cum.rename(columns={c: c2 for c, c2 in zip(cum.columns, 
                                           ['date', 'type', 'kind', 'count'])}, 
           inplace=True)
cum

In [ ]:
sns.relplot(
    data=cum,
    kind='line',
    x='date',
    y='count',
    col='kind',
    hue='type'
)

# Geography

In [ ]:
tmp = cum[cum.type.isin(['U', 'DL', 'LDNR', 'total'])]
ax = sns.relplot(
    data=tmp,
    kind='line',
    x='date',
    y='count',
    col='kind',
    hue='type'
)
ax.figure.set_size_inches(9.5, 6)
ax.set_xticklabels(rotation=30)
plt.suptitle('Cumulative count');

In [ ]:
import plotly.express as plex

In [ ]:
plex.bar(
    tmp[(tmp.type!='total') & (tmp.kind == 'killed')], 
    x='date', 
    y='count',
    color='type',
    barmode='group'
)

# Gender

In [ ]:
tmp = cum[cum.type.isin(['men', 'boys', 'women', 'girls', 'total'])].copy()
tmp['gender'] = 'u'
tmp.loc[tmp.type.isin(['men', 'boys']), 'gender'] = 'm'
tmp.loc[tmp.type.isin(['women', 'girls']), 'gender'] = 'f'
ax = sns.relplot(
    data=tmp,
    kind='line',
    x='date',
    y='count',
    col='kind',
    hue='type',
    row='gender'
)
ax.figure.set_size_inches(9.5, 6)
ax.set_xticklabels(rotation=30)
plt.suptitle('Cumulative count');

# Dash

In [ ]:
import plotly.express as px

In [ ]:
df = unhr.data.dropna().stack(level=0).reset_index().rename(columns={'level_0':'date', 'level_1':'kind'})
#df

In [ ]:
import ipywidgets as ui

data = unhr.data.dropna().stack(level=0).reset_index().rename(columns={'level_0':'date', 'level_1':'kind'})

@ui.interact
def bla(p=['total', 'geo', 'gen', 'age', 'gen-age', 'gen-age2'], 
        view=ui.RadioButtons(options=['cumulative', 'daily', '7D MA'])):
    df = unhr.data
    if view in ['daily', '7D MA']:
        df = df.loc[:df.last_valid_index()].astype(float).interpolate().diff()
        if view == '7D MA':
            df = df.rolling('7D').mean()
    df = df.dropna().stack(level=0).reset_index().rename(columns={'level_0': 'date', 'level_1': 'kind'})
    dk = ['date', 'kind']
    cols_geo = ['DL', 'U', 'LDNR']
    cols_gender = {'m': ['men', 'boys'], 'f': ['women', 'girls'], 'u': ['adults', 'children']}
    cols_age = {'adults_total': ['men', 'women', 'adults'] , 'children_total': ['boys', 'girls', 'children']}
    if p == 'total':
        total = unhr.data.T.xs('total', level=1).T
        total = total.loc[total.last_valid_index()]
        return px.line(df, x='date', y='total', color='kind', 
                       title=f'Total {total.killed:,} killed and {total.injured:,} injured')
    elif p == 'geo':
        ddf = df[dk + cols_geo]
        return px.line(ddf, x='date', y=cols_geo, facet_col='kind')
    elif p == 'gen':
        ddf = df[dk + [ai for a in list(cols_gender.values()) for ai in a]].copy()
        for g, c in cols_gender.items():
            ddf[g] = ddf[c].sum(axis=1)
        ddf = ddf[dk + list(cols_gender)]
        return px.line(ddf, x='date', y=cols_gender, facet_col='kind')
    elif p == 'age':
        ddf = df[dk + [ai for a in list(cols_age.values()) for ai in a]].copy()
        for g, c in cols_age.items():
            ddf[g] = ddf[c].sum(axis=1)
        ddf = ddf[dk + list(cols_age)]
        return px.line(ddf, x='date', y=cols_age, facet_col='kind')        
    elif p == 'gen-age':
        ddf = df[dk + [ai for a in list(cols_gender.values()) for ai in a]]
        ddf = ddf.set_index(['date', 'kind']).stack().reset_index().rename(columns={'level_2': 'type', 0: 'value'})
        ddf['gender'] = None
        for g, c in cols_gender.items():
            ddf.loc[ddf['type'].isin(c), 'gender'] = g
        ddf['age'] = None
        for g, c in cols_age.items():
            ddf.loc[ddf['type'].isin(c), 'age'] = g.split('_')[0]
        return px.line(ddf, x='date', y='value', facet_col='kind', facet_row='age', color='gender')
    elif p == 'gen-age2':
        ddf = df[dk + [ai for a in list(cols_gender.values()) for ai in a]]
        ddf = ddf.set_index(['date', 'kind']).stack().reset_index().rename(columns={'level_2': 'type', 0: 'value'})
        ddf['gender'] = None
        for g, c in cols_gender.items():
            ddf.loc[ddf['type'].isin(c), 'gender'] = g
        return px.line(ddf, x='date', y='value', facet_col='kind', facet_row='gender', color='type')
    else:
        return f'Not implemented for {p}'